In [ ]:
import os, sys 
import torch 
import datetime
from pprint import pprint
import matplotlib.pyplot as plt
import numpy as np
from pprint import pprint
import IPython.display as ipd
from argparse import Namespace
import torch.nn as nn 
import torch.nn.functional as F
import librosa
import argparse
import scipy 
from IPython.display import Audio
import numpy as np
from matplotlib.pyplot import cm
from tqdm import tqdm

sys.path.append("../../")

In [ ]:
from __config__ import *
import two_step_mask_learning.dnn.models.conv_tasnet_maskregress as mask_tasnet
import two_step_mask_learning.dnn.models.simplified_tasnet as simple_tasnet
import two_step_mask_learning.dnn.dataset_loader.augmented_mix_dataloader as augmented_dataloader
import two_step_mask_learning.dnn.dataset_loader.torch_dataloader as wsjnormpad_loader


In [ ]:
import two_step_mask_learning.dnn.losses.sisdr as sisdr_lib
import two_step_mask_learning.dnn.models.adaptive_frontend as adaptive_fe

In [ ]:
def sort_by
def zero_crossings(m):
    signs = np.sign(m)
    zcs = np.abs(signs[1:,:] - signs[:-1,:]) / 2.
    return zcs

n_time_samples, n_freqs = enc_basis.T.shape
zero_cross_per_col = zero_crossings(enc_basis.T)
e = enc_basis.T[:, np.argsort(np.sum(zero_cross_per_col, axis=0))]
new_bias = enc_bias[np.argsort(np.sum(zero_cross_per_col, axis=0))]


In [ ]:
mask_model_path = '/home/thymios/tn_mask/augmented/ESC50_1.0/'
a_mask_model = mask_tasnet.CTN.load_best_model(mask_model_path, 'softmax', 128, 21, 0.)
mask_enc = a_mask_model.encoder.conv.weight.detach().cpu().squeeze().numpy().T
mask_dec = a_mask_model.decoder.deconv.weight.detach().cpu().squeeze().numpy().T
# print the encoder 
plt.figure(figsize=(20,10))
im = plt.imshow(masc_enc, interpolation='nearest')
plt.colorbar(im, fraction=0.005)
plt.show()

plt.figure(figsize=(20,10))
im = plt.imshow(mask_dec[:, :mask_dec.shape[-1]//2], interpolation='nearest')
plt.colorbar(im, fraction=0.005)
plt.show()

In [ ]:
simple_tasnet_model_path = '/home/thymios/tn_mask/augmented/WSJMIX_0.5_ESC50_0.5/'
a_tasnet_model = simple_tasnet.GLNFullThymiosCTN.load_best_model(simple_tasnet_model_path, 128, 21)
mask_enc = a_tasnet_model.fe[0].weight.detach().cpu().squeeze().numpy().T
mask_dec = a_tasnet_model.be.weight.detach().cpu().squeeze().numpy().T
# print the encoder 
plt.figure(figsize=(20,10))
im = plt.imshow(mask_enc, interpolation='nearest')
plt.colorbar(im, fraction=0.005)
plt.show()

plt.figure(figsize=(20,10))
im = plt.imshow(mask_dec[:, :mask_dec.shape[-1]//2], interpolation='nearest')
plt.colorbar(im, fraction=0.005)
plt.show()

In [ ]:
# Try to do inference with different loaders
n_samples = 2
batch_size = 2

# wsj_normpad_gen = wsjnormpad_loader.get_data_generators(
#             [WSJ_MIX_2_8K_PREPROCESSED_EVAL_P], bs=batch_size,
#             n_jobs=4, get_top=[n_samples],
#             return_items=['mixture_wav_norm', 'clean_sources_wavs_norm'])[0]

these_args = argparse.Namespace(
            input_dataset_p=[WSJ_MIX_HIERARCHICAL_P + '/test/', ESC50_HIERARCHICAL_P + '/test/'],
            datasets_priors=[0.5, 0.5],
            batch_size=batch_size,
            n_jobs=4,
            n_samples=n_samples,
            return_items=['wav'],
            fs=8000.,
            selected_timelength=4.,
            n_sources=2,
            max_abs_snr=2.5,
            fixed_seed=12
        )

data_loader = augmented_dataloader.AugmentedOnlineMixingDataset(
    **vars(these_args))
wsj_half_esc50_gen = augmented_dataloader.get_data_gen_from_loader(data_loader)

these_args = argparse.Namespace(
            input_dataset_p=[ESC50_HIERARCHICAL_P + '/test/'],
            datasets_priors=[1.],
            batch_size=batch_size,
            n_jobs=4,
            n_samples=n_samples,
            return_items=['wav'],
            fs=8000.,
            selected_timelength=4.,
            n_sources=2,
            max_abs_snr=2.5,
            fixed_seed=8
        )

data_loader = augmented_dataloader.AugmentedOnlineMixingDataset(
    **vars(these_args))
esc50_gen = augmented_dataloader.get_data_gen_from_loader(data_loader)

these_args = argparse.Namespace(
            input_dataset_p=[WSJ_MIX_HIERARCHICAL_P + '/test/'],
            datasets_priors=[1.],
            batch_size=batch_size,
            n_jobs=4,
            n_samples=n_samples,
            return_items=['wav'],
            fs=8000.,
            selected_timelength=4.,
            n_sources=2,
            max_abs_snr=2.5,
            fixed_seed=8
        )

data_loader = augmented_dataloader.AugmentedOnlineMixingDataset(
    **vars(these_args))
wsj_gen = augmented_dataloader.get_data_gen_from_loader(data_loader)

In [ ]:
# Get some good files for showing their latent representations
selected_ind = 0
for i, (mixture, sources) in enumerate(wsj_half_esc50_gen):
    if selected_ind == i:
        mix_wav = mixture.squeeze()[0]
        s1_wav = sources.squeeze()[0][0]
        s2_wav = sources.squeeze()[0][1]
        print('Mixture')
        ipd.display(Audio(mix_wav, rate=8000))
        print('Source 1')
        ipd.display(Audio(s1_wav, rate=8000))
        print(mix_wav.shape)
        print('Source 2')
        ipd.display(Audio(s2_wav, rate=8000))
        print(s1_wav.shape)
        break

In [ ]:
def plot_encoded(mix_encoded, s1_encoded, s2_encoded, exponent=0.003):
    plt.figure(figsize=(100,10))
    plt.imshow(mix_encoded**exponent, cmap=cm.Greens_r)
    plt.title('Mixture encoded', fontsize=40)
    plt.show()

#     s1_encoded -= np.mean(s1_encoded, axis=1, keepdims=True)
#     mix_encoded -= np.mean(mix_encoded, axis=1, keepdims=True)
#     s2_encoded -= np.mean(s2_encoded, axis=1, keepdims=True)

#     s1_encoded = np.abs(s1_encoded)
#     mix_encoded = np.abs(mix_encoded)
#     s2_encoded = np.abs(s2_encoded)
    
    plt.figure(figsize=(100,10))
    plt.imshow(s1_encoded**exponent, interpolation='nearest', cmap=cm.Blues_r)
    plt.title('Source 1 Mask', fontsize=40)
    plt.show()

    plt.figure(figsize=(100,10))
    plt.imshow(s2_encoded**exponent, interpolation='nearest', cmap=cm.Reds_r)
    plt.title('Source 2 Mask', fontsize=40)
    plt.show()
    
    plt.figure(figsize=(100,10))
    plt.imshow(abs(s2_encoded - s1_encoded)**exponent , interpolation='nearest', cmap=cm.Reds_r)
    plt.title('Source 2 - Source 1', fontsize=40)
    plt.show()
    
    plt.figure(figsize=(100,10))
    plt.imshow((s1_encoded )**exponent , interpolation='nearest', cmap=cm.Blues_r, alpha=0.9)
    plt.imshow((s2_encoded)**exponent , interpolation='nearest', cmap=cm.Reds_r, alpha=0.6)
    plt.title('Superposition', fontsize=40)
    plt.show()
    
    
def better_plot_encoded(mix_encoded, s1_encoded, s2_encoded, exponent=0.003):
#     fig, (ax1, ax2, ax3) = plt.subplots(nrows=1, ncols=3)
    fig = plt.figure(figsize=(20,10))
    ax1 = fig.add_subplot(1, 3, 1)
    ax2 = fig.add_subplot(1, 3, 2)
    ax3 = fig.add_subplot(1, 3, 3)

    title_size = 24
    
    ax1.set_title('Mixture', fontsize=title_size)
    ax2.set_title('Speech', fontsize=title_size)
    ax3.set_title('Bird Chirping', fontsize=title_size)
    
    aspect = 0.00375
    ax1.imshow(mix_encoded.squeeze()**exponent, interpolation='none', 
               cmap=cm.Greens_r, extent=[0,1,0,127], aspect=aspect)
    ax1.xaxis.set_visible(False)
    ax1.set_ylabel('Encoder Basis Index', fontsize=19)

#     ax1.set_yticks(np.arange(mix_encoded.shape[0]))
#     ax1.yaxis.set_visible(False)

    
    ax2.imshow(s1_encoded.squeeze()**exponent, interpolation='none', cmap=cm.Blues_r,
              extent=[0,1,0,127], aspect=aspect)
    ax2.yaxis.set_visible(False)
    ax2.xaxis.set_visible(False)

    
    ax3.imshow(s2_encoded.squeeze()**exponent, interpolation='none', cmap=cm.Reds_r,
              extent=[0,1,0,127], aspect=aspect)
    ax3.yaxis.set_visible(False)
    ax3.xaxis.set_visible(False)

    plt.show()
    fig.savefig("our_latents.pdf", bbox_inches='tight')

In [ ]:
mask_model_path = '/home/thymios/tn_mask/augmented/WSJMIX_0.5_ESC50_0.5/'
a_mask_model = mask_tasnet.CTN.load_best_model(mask_model_path, 'softmax', 128, 21, 0.)
mask_enc = a_mask_model.encoder
mask_dec = a_mask_model.decoder

enc_basis = mask_enc.conv.weight.squeeze().numpy()
enc_bias = mask_enc.conv.bias.squeeze().numpy()

e = enc_basis.T[:, np.argsort(np.sum(np.abs(enc_basis.T), axis=0))]
new_bias = enc_bias[np.argsort(np.sum(np.abs(enc_basis.T), axis=0))]

mask_enc.conv.weight = torch.nn.Parameter(torch.tensor(e.T).unsqueeze(1))
mask_enc.conv.bias = torch.nn.Parameter(torch.tensor(new_bias).squeeze())

offset = 11000
duration = 8000

mix_encoded = mask_enc(torch.tensor(mix_wav[offset:offset+duration]).unsqueeze(0).unsqueeze(0)).detach().numpy()
s1_encoded = mask_enc(torch.tensor(s1_wav[offset:offset+duration]).unsqueeze(0).unsqueeze(0)).detach().numpy()
s2_encoded = mask_enc(torch.tensor(s2_wav[offset:offset+duration]).unsqueeze(0).unsqueeze(0)).detach().numpy()
# plot_encoded(mix_encoded.squeeze(), s1_encoded.squeeze(), s2_encoded.squeeze(), exponent=0.1)

better_plot_encoded(mix_encoded, s1_encoded, s2_encoded, exponent=0.1)

# Measure the sparsity 
print("L1 sparsity")
print(np.abs(s1_encoded - np.mean(s1_encoded, axis=2, keepdims=True)).sum(),
      np.abs(s2_encoded - np.mean(s2_encoded, axis=2, keepdims=True)).sum(),
      np.abs(mix_encoded - np.mean(mix_encoded, axis=2, keepdims=True)).sum())
print("L0 sparsity")
print(np.abs(s1_encoded - np.mean(s1_encoded, axis=2, keepdims=True)).sum(),
      np.abs(s2_encoded - np.mean(s2_encoded, axis=2, keepdims=True)).sum(),
      np.abs(mix_encoded - np.mean(mix_encoded, axis=2, keepdims=True)).sum())
print(1. * np.count_nonzero(np.abs(s1_encoded - np.mean(s1_encoded, axis=2, keepdims=True))) / np.prod(s1_encoded.shape))
print(1. * np.count_nonzero(np.abs(s2_encoded - np.mean(s2_encoded, axis=2, keepdims=True))) / np.prod(s1_encoded.shape))
print(1. * np.count_nonzero(np.abs(mix_encoded - np.mean(mix_encoded, axis=2, keepdims=True))) / np.prod(s1_encoded.shape))


In [ ]:
def better_plot_encoded_tasnet(mix_encoded, s1_encoded, s2_encoded, exponent=0.003):
#     fig, (ax1, ax2, ax3) = plt.subplots(nrows=1, ncols=3)
    fig = plt.figure(figsize=(20,10))
    ax1 = fig.add_subplot(1, 3, 1)
    ax2 = fig.add_subplot(1, 3, 2)
    ax3 = fig.add_subplot(1, 3, 3)

#     ax1.set_title('Mixture', fontsize=20)
#     ax2.set_title('Speech', fontsize=20)
#     ax3.set_title('Toothbrush Sound', fontsize=20)
    
    aspect = 0.00375
    ax1.imshow(mix_encoded.squeeze()**exponent, interpolation='none', 
               cmap=cm.Greens_r, extent=[0,1,0,127], aspect=aspect)
    ax1.set_xlabel('Time (secs)', fontsize=20)
    ax1.set_ylabel('Encoder Basis Index', fontsize=19)

#     ax1.xaxis.set_visible(False)
#     ax1.set_yticks(np.arange(mix_encoded.shape[0]))
#     ax1.yaxis.set_visible(False)

    
    ax2.imshow(s1_encoded.squeeze()**exponent, interpolation='none', cmap=cm.Blues_r,
              extent=[0,1,0,127], aspect=aspect)
    ax2.yaxis.set_visible(False)
    ax2.set_xlabel('Time (secs)', fontsize=20)

#     ax2.xaxis.set_visible(False)

    
    ax3.imshow(s2_encoded.squeeze()**exponent, interpolation='none', cmap=cm.Reds_r,
              extent=[0,1,0,127], aspect=aspect)
    ax3.yaxis.set_visible(False)
#     ax3.xaxis.set_visible(False)
    ax3.set_xlabel('Time (secs)', fontsize=20)


    plt.show()
    fig.savefig("tasnet_latents.pdf", bbox_inches='tight')

In [ ]:
# do the same for the trained tasnet
simple_tasnet_model_path = '/home/thymios/tn_mask/augmented/WSJMIX_0.5_ESC50_0.5/'
a_tasnet_model = simple_tasnet.GLNFullThymiosCTN.load_best_model(simple_tasnet_model_path, 128, 21)
# mask_enc = a_mask_model.encoder
# mask_dec = a_mask_model.decoder

enc_basis = a_tasnet_model.fe[0].weight.squeeze().detach().cpu().numpy()
enc_bias = a_tasnet_model.fe[0].bias.squeeze().detach().cpu().numpy()

e = enc_basis.T[:, np.argsort(np.sum(np.abs(enc_basis.T), axis=0))]
new_bias = enc_bias[np.argsort(np.sum(np.abs(enc_basis.T), axis=0))]

mask_enc.conv.weight = torch.nn.Parameter(torch.tensor(e.T).unsqueeze(1))
mask_enc.conv.bias = torch.nn.Parameter(torch.tensor(new_bias).squeeze())

offset = 11000
duration = 8000

mix_encoded = mask_enc(torch.tensor(mix_wav[offset:offset+duration]).unsqueeze(0).unsqueeze(0)).detach().numpy()
s1_encoded = mask_enc(torch.tensor(s1_wav[offset:offset+duration]).unsqueeze(0).unsqueeze(0)).detach().numpy()
s2_encoded = mask_enc(torch.tensor(s2_wav[offset:offset+duration]).unsqueeze(0).unsqueeze(0)).detach().numpy()
# plot_encoded(mix_encoded.squeeze(), s1_encoded.squeeze(), s2_encoded.squeeze(), exponent=0.1)
better_plot_encoded_tasnet(mix_encoded, s1_encoded, s2_encoded, exponent=0.1)

# Measure the sparsity 
print("L1 sparsity")
print(np.abs(s1_encoded - np.mean(s1_encoded, axis=2, keepdims=True)).sum(),
      np.abs(s2_encoded - np.mean(s2_encoded, axis=2, keepdims=True)).sum(),
      np.abs(mix_encoded - np.mean(mix_encoded, axis=2, keepdims=True)).sum())
print("L0 sparsity")
print(np.abs(s1_encoded - np.mean(s1_encoded, axis=2, keepdims=True)).sum(),
      np.abs(s2_encoded - np.mean(s2_encoded, axis=2, keepdims=True)).sum(),
      np.abs(mix_encoded - np.mean(mix_encoded, axis=2, keepdims=True)).sum())
print(1. * np.count_nonzero(np.abs(s1_encoded - np.mean(s1_encoded, axis=2, keepdims=True))) / np.prod(s1_encoded.shape))
print(1. * np.count_nonzero(np.abs(s2_encoded - np.mean(s2_encoded, axis=2, keepdims=True))) / np.prod(s1_encoded.shape))
print(1. * np.count_nonzero(np.abs(mix_encoded - np.mean(mix_encoded, axis=2, keepdims=True))) / np.prod(s1_encoded.shape))


In [ ]:
# Evaluate on WSJnorm pad
n_fft = 256
hop_length = int(n_fft / 4.)
win_length = n_fft

def evaluate_ibm_on_generator(loss_func, generator, generator_name='WSJ'):
    acc = []
    with torch.no_grad():
        for data in tqdm(generator, desc='Validation on {}'.format(generator_name)):
            m1wavs = data[0].squeeze().detach().numpy()
            clean_wavs = data[-1].squeeze().detach().numpy()
            
            m1 = m1wavs
            
            s1 = clean_wavs[0]
            s2 = clean_wavs[1]
            
            m1_stft = librosa.core.stft(m1, n_fft=n_fft, hop_length=hop_length, win_length=win_length)
            s1_stft = librosa.core.stft(s1, n_fft=n_fft, hop_length=hop_length, win_length=win_length)
            s2_stft = librosa.core.stft(s2, n_fft=n_fft, hop_length=hop_length, win_length=win_length)
            s1_energy = np.abs(s1_stft)**2
            s2_energy = np.abs(s2_stft)**2

            irm1_stft = np.abs(s1_stft) / (np.abs(s2_stft) + np.abs(s1_stft) + 10e-9)
            irm2_stft = np.abs(s2_stft) / (np.abs(s2_stft) + np.abs(s1_stft) + 10e-9)

            s1_pred_stft = m1_stft * irm1_stft
            s2_pred_stft = m1_stft * irm2_stft

            s1_pred = librosa.core.istft(s1_pred_stft, hop_length=hop_length, win_length=win_length)
            s2_pred = librosa.core.istft(s2_pred_stft, hop_length=hop_length, win_length=win_length)

            pred_sources = np.stack([s1_pred, s2_pred])
            pred_sources = torch.from_numpy(pred_sources)
         
            l = loss_func(pred_sources.unsqueeze(0), data[-1], initial_mixtures=data[0].unsqueeze(1))
            acc += l.tolist()
    return acc 

sisdr_loss = sisdr_lib.PermInvariantSISDR(batch_size=batch_size,
                                          n_sources=2,
                                          zero_mean=True,
                                          backward_loss=False,
                                          improvement=True,
                                          return_individual_results=True)

In [ ]:
# Try to estimate IBM for all test sets
transfer_generators_dict = {'WSJ': wsj_normpad_gen, 'WSJ.5ESC.5': wsj_half_esc50_gen, 'ESC': esc50_gen}
transfer_results_dict = {}

for generator_name, generator in transfer_generators_dict.items():
    results = evaluate_ibm_on_generator(sisdr_loss, generator, generator_name=generator_name)
    results = np.array(results)
    transfer_results_dict[generator_name] = {}
    transfer_results_dict[generator_name]['mean'] = results.mean()
    transfer_results_dict[generator_name]['std'] = results.std()
    pprint(transfer_results_dict)
pprint(transfer_results_dict)